# Week4 授業課題

___
## 問題２：学習と検証

### import section

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

### read csv

In [3]:
train_dataset = pd.read_csv("../Week3/home-credit-default-risk/application_train.csv")
test_dataset = pd.read_csv("../Week3/home-credit-default-risk/application_test.csv")

# train_dataset.info()

### データの前処理

In [4]:
# Concatenate two dataset to treat as one
train_test_df = pd.concat([train_dataset, test_dataset], axis=0, sort=False)

# Fill missings
train_test_df = train_test_df.fillna(-1)

# Replace categorical value to dummy value
train_test_df = pd.get_dummies(train_test_df)

# Devide concatenated dataset as original form each dataset
train_dataset_nomiss = train_test_df.iloc[:307511, :]
test_dataset_nomiss = train_test_df.iloc[307511:, :]

# Create train data and test data from train_dataset_nomiss
x_train, x_test, y_train, y_test = train_test_split(np.array(train_dataset_nomiss.iloc[:, 2:]), \
                                                    np.array(train_dataset_nomiss.loc[:, ["TARGET"]]), \
                                                    train_size=0.70, test_size=0.20)
                                                                                                                                                              

In [ ]:
sc = StandardScaler()
sc.fit(x_train)

x_train_trans = sc.transform(x_train)
x_test_trans = sc.transform(x_test)

model = KNeighborsClassifier(n_neighbors=1)
model.fit(x_train_trans, y_train)


In [ ]:
model.predict_proba(x_test_trans)

In [ ]:
model.score(x_test_trans, y_test)